In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,auc,confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, roc_auc_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTENC, SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from imblearn.metrics import specificity_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [2]:
def rf_models(cov, n_boots = 10):
    best_params = []
    n_ssi = sum(cov["ssi"] == 1)
    rf = []
    df = cov
#     n_boots = 100
    #cov_train_final = cov
    
    '''cat_names = []
    for i, j in enumerate(cov_train_final.loc[:, ~cov_train_final.columns.isin(["ssi"])].columns):
        if cov_train_final[j].dtypes.name == "category":
            cat_names.append(i)'''
    
    for i in range(n_boots):
        df = cov.groupby("ssi").apply(pd.DataFrame.sample, n = int(n_ssi))
        #smote = SMOTE(sampling_strategy = 0.4, random_state = 42)
        #cov_train_feat_smote, cov_train_target_smote = smote.fit_resample(cov_train_final.loc[:, ~cov_train_final.columns.isin(["ssi"])], 
         #                                                                 cov_train_final["ssi"])
        #random_under = RandomUnderSampler(sampling_strategy = 0.6)
        #cov_train_feat_rus, cov_train_target_rus = random_under.fit_sample(cov_train_feat_smote, cov_train_target_smote)

        forest = RandomForestClassifier(n_estimators = 500, verbose=0, random_state = 42, n_jobs = -1)
        parameters = {'min_samples_leaf' : [1, 2], 'max_features' : ["auto", "log2", 3, 5],#,"log2", 3, 5, 7, 10],
                     'min_samples_split': [2] #+ list(np.linspace(0.005, 0.02, 4, endpoint=True)) + 
                      #, 'class_weight' : ["balanced", {0:1, 1:30}]
                      ,'max_depth': [None, 6, 10]
                     }
        #score = ["precision", "recall", "roc_auc"]
        score = ["roc_auc"]
        #model = RandomForestClassifier(n_estimators = 500, verbose=0, min_samples_leaf = min_leaf, max_features = 3, 
         #                              min_samples_split = min_split, random_state=i, 
         #                              class_weight = {0:0.2,1:50}, n_jobs = -1)
        #model = RandomForestClassifier(n_estimators = 500, verbose=0, random_state=i, n_jobs = -1)

        grid = GridSearchCV(forest, param_grid = parameters, scoring = score, refit = "roc_auc")
        grid.fit(df.loc[:,~df.columns.isin(["ssi"])], df["ssi"])
        #grid.fit(cov_train_feat_rus, cov_train_target_rus)
        model = grid.best_estimator_
        best_params.append(list(grid.best_params_.values()))
        best_score = grid.best_score_
        #model.fit(df.loc[:,~df.columns.isin(["ssi"])], df["ssi"])
        rf.append(model)
    return rf, best_params, best_score


#mi = np.linspace(0.0067, 0.01, 4, endpoint=True)
#for min_split in [0.1, 0.2, 0.3]:
#for min_leaf in [8, 10, 12]:



In [ ]:
def lr_models(cov):
    
    scaler = StandardScaler()
    X = cov.loc[:,~cov.columns.isin(["ssi"])]
    scaler.fit(X)
    X = scaler.transform(X)
    Y = cov["ssi"]
    
    lr  = LogisticRegression(max_iter = 500, random_state = 42, n_jobs = -1)
    parameters = {'C' : [1e-7, 1e-6, 1e-5, 1e-4, 5e-4],
                 'class_weight' : ["balanced", {0:1, 1:30}]}
    #score = ["precision", "recall", "roc_auc"]
    score = ["roc_auc"]
    grid = GridSearchCV(lr, param_grid = parameters, scoring = score, refit = "roc_auc")
    grid.fit(X, Y)
    
    model = grid.best_estimator_
    return model, list(grid.best_params_.values())
    

In [ ]:
def sgd_models(cov):
    
    scaler = StandardScaler()
    X = cov.loc[:,~cov.columns.isin(["ssi"])]
    scaler.fit(X)
    X = scaler.transform(X)
    Y = cov["ssi"]
    
    lr  = SGDClassifier(loss = 'log', max_iter = 800, random_state = 42, n_jobs = -1)
    parameters = {'alpha' : [1e-6, 1e-5, 1e-4, 5e-4]}
    #score = ["precision", "recall", "roc_auc"]
    score = ["roc_auc"]
    grid = GridSearchCV(lr, param_grid = parameters, scoring = score, refit = "roc_auc")
    grid.fit(X, Y)
    
    model = grid.best_estimator_
    return model, list(grid.best_params_.values())

In [14]:
def xg_models(cov):

    param = {'booster':['gbtree'], 'max_depth': [2, 4], 'objective': ['binary:logistic'], 
             'use_label_encoder': [False], 'max_delta_step': [1,3], 'eval_metric' : ['auc']
            , 'subsample': [0.8,0.6]
             , 'colsample_bytree': [0.7, 0.5]}
    X = cov.loc[:,~cov.columns.isin(["ssi"])]
    Y = np.array(cov["ssi"])
    clf = XGBClassifier()
    score = ["roc_auc"]
    grid = GridSearchCV(clf, param_grid = param, scoring = score, refit = "roc_auc")
    grid.fit(X, Y)
    model = grid.best_estimator_
    return model, list(grid.best_params_.values())

In [ ]:
def qda_models(cov):
    clf = QuadraticDiscriminantAnalysis()
    param = {'reg_param': np.arange(0, 1, 0.2)}
    scaler = StandardScaler()
    X = cov.loc[:,~cov.columns.isin(["ssi"])]
    scaler.fit(X)
    X = scaler.transform(X)
    Y = cov["ssi"]
    score = ["roc_auc"]
    grid = GridSearchCV(clf, param_grid = param, scoring = score, refit = "roc_auc")
    grid.fit(X, Y)
    model = grid.best_estimator_
    return model